# First Best-Fit Line

***

## Project Description

## Import Libraries

In [1]:
import numpy as np
from numpy import count_nonzero
from numpy import median
from numpy import mean
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import random

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

import datetime
from datetime import datetime, timedelta

#import os
#import zipfile
import scipy.stats
from collections import Counter

# import pandas_profiling
# from pandas_profiling import ProfileReport

import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet, Lasso, Ridge
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, auc, classification_report, confusion_matrix, f1_score
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve

%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', titlesize=9)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings('ignore')

#Webscraping
#import requests
#from bs4 import BeautifulSoup

# Use Folium library to plot values on a map.
#import folium

# Use Feature-Engine library
#import feature_engine
#import feature_engine.missing_data_imputers as mdi
#from feature_engine.outlier_removers import Winsorizer
#from feature_engine import categorical_encoders as ce
#from feature_engine.discretisation import EqualWidthDiscretiser, EqualFrequencyDiscretiser, DecisionTreeDiscretiser
#from feature_engine.encoding import OrdinalEncoder

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


## Merge tables

In [2]:
df = pd.read_csv("rentaldate.csv")
df

,rental_date,st_property_id
0,31/1/2014,ST100
1,31/1/2014,ST106
2,31/1/2014,ST107
3,31/1/2014,ST109
4,31/1/2014,ST113
...,...,...
223007,31/12/2015,ST995
223008,31/12/2015,ST996
223009,31/12/2015,ST997
223010,31/12/2015,ST998


In [3]:
df2 = pd.read_csv("propertyinfo.csv")
df2

,st_property_id,location,property_type
0,ST1,L9531,R6
1,ST10,L9533,R6
2,ST100,L1944,R2
3,ST1000,L5957,R1
4,ST101,L15257,R2
...,...,...,...
994,ST995,L5458,R4
995,ST996,L10771,R4
996,ST997,L11671,R12
997,ST998,L17570,R1


In [4]:
df3 = pd.merge(left=df, right=df2, on="st_property_id", how="inner")

In [5]:
df3

,rental_date,st_property_id,location,property_type
0,31/1/2014,ST100,L1944,R2
1,3/2/2014,ST100,L1944,R2
2,7/2/2014,ST100,L1944,R2
3,11/2/2014,ST100,L1944,R2
4,13/2/2014,ST100,L1944,R2
...,...,...,...,...
223007,27/12/2015,ST929,L14897,R16
223008,28/12/2015,ST929,L14897,R16
223009,29/12/2015,ST929,L14897,R16
223010,30/12/2015,ST929,L14897,R16


In [6]:
#df3.to_csv("merge1.csv",index=False)

In [7]:
df4 = pd.read_csv("watershed.csv")
df4

,ws_property_id,location,property_type_id,current_monthly_rent
0,W1,L9531,R6,1060
1,W10,L9533,R6,1200
2,W100,L1944,R2,3300
3,W101,L15257,R2,1400
4,W102,L15257,R6,2000
...,...,...,...,...
239,W95,L1943,R10,4000
240,W96,L1943,R14,5100
241,W97,L1944,R6,5600
242,W98,L1944,R10,5000


In [8]:
df5 = pd.read_csv("propertytype.csv")
df5

,property_type_id,apt_house,num_bedrooms,kitchen,shared
0,R1,apartment,1,Y,Y
1,R10,house,1,Y,N
2,R11,house,1,N,Y
3,R12,house,1,N,N
4,R13,house,2,Y,Y
5,R14,house,2,Y,N
6,R15,house,2,N,Y
7,R16,house,2,N,N
8,R2,apartment,1,Y,N
9,R3,apartment,1,N,Y


In [9]:
df6 = pd.merge(left=df4, right=df5, on="property_type_id", how="inner")

In [10]:
df6

,ws_property_id,location,property_type_id,current_monthly_rent,apt_house,num_bedrooms,kitchen,shared
0,W1,L9531,R6,1060,apartment,2,Y,N
1,W10,L9533,R6,1200,apartment,2,Y,N
2,W102,L15257,R6,2000,apartment,2,Y,N
3,W106,L15260,R6,1900,apartment,2,Y,N
4,W110,L15264,R6,1300,apartment,2,Y,N
...,...,...,...,...,...,...,...,...
239,W83,L1940,R14,5500,house,2,Y,N
240,W87,L1941,R14,5000,house,2,Y,N
241,W91,L1942,R14,3500,house,2,Y,N
242,W96,L1943,R14,5100,house,2,Y,N


In [11]:
#df6.to_csv("merge2.csv",index=False)

In [12]:
df7 = pd.read_csv("rentalprice.csv")
df7

,location,property_type_id,percentile_10th_price,percentile_90th_price,sample_nightly_rent_price
0,L10126,R1,83,273,192
1,L10126,R10,155,494,257
2,L10126,R11,82,411,242
3,L10126,R12,119,366,229
4,L10126,R13,137,416,375
...,...,...,...,...,...
971,L9534,R5,69,151,115
972,L9534,R6,111,147,123
973,L9534,R7,69,139,104
974,L9534,R8,101,199,163


In [13]:
df7.drop(['location'], axis=1, inplace=True)

In [14]:
df7.head()

,property_type_id,percentile_10th_price,percentile_90th_price,sample_nightly_rent_price
0,R1,83,273,192
1,R10,155,494,257
2,R11,82,411,242
3,R12,119,366,229
4,R13,137,416,375


In [15]:
df9 = pd.merge(left=df6, right=df7, on="property_type_id", how="inner")

In [16]:
df9

,ws_property_id,location,property_type_id,current_monthly_rent,apt_house,num_bedrooms,kitchen,shared,percentile_10th_price,percentile_90th_price,sample_nightly_rent_price
0,W1,L9531,R6,1060,apartment,2,Y,N,168,392,322
1,W1,L9531,R6,1060,apartment,2,Y,N,154,286,230
2,W1,L9531,R6,1060,apartment,2,Y,N,192,357,266
3,W1,L9531,R6,1060,apartment,2,Y,N,94,531,427
4,W1,L9531,R6,1060,apartment,2,Y,N,154,480,154
...,...,...,...,...,...,...,...,...,...,...,...
14883,W99,L1944,R14,6000,house,2,Y,N,192,829,575
14884,W99,L1944,R14,6000,house,2,Y,N,97,240,199
14885,W99,L1944,R14,6000,house,2,Y,N,136,335,188
14886,W99,L1944,R14,6000,house,2,Y,N,136,336,186


In [17]:
df9.duplicated().sum()

0

In [18]:
#df9.to_csv("merge3.csv",index=False)

## Exploratory Data Analysis

In [19]:
df = pd.read_csv("merge3.csv")
df

,ws_property_id,location,property_type_id,current_monthly_rent,apt_house,num_bedrooms,kitchen,shared,percentile_10th_price,percentile_90th_price,sample_nightly_rent_price
0,W1,L9531,R6,1060,apartment,2,Y,N,168,392,322
1,W1,L9531,R6,1060,apartment,2,Y,N,154,286,230
2,W1,L9531,R6,1060,apartment,2,Y,N,192,357,266
3,W1,L9531,R6,1060,apartment,2,Y,N,94,531,427
4,W1,L9531,R6,1060,apartment,2,Y,N,154,480,154
...,...,...,...,...,...,...,...,...,...,...,...
14883,W99,L1944,R14,6000,house,2,Y,N,192,829,575
14884,W99,L1944,R14,6000,house,2,Y,N,97,240,199
14885,W99,L1944,R14,6000,house,2,Y,N,136,335,188
14886,W99,L1944,R14,6000,house,2,Y,N,136,336,186


## Regression Analysis

### Linear Regression (StatsModel)

In [20]:
df.columns

Index(['ws_property_id', 'location', 'property_type_id', 'current_monthly_rent', 'apt_house', 'num_bedrooms', 'kitchen', 'shared', 'percentile_10th_price', 'percentile_90th_price', 'sample_nightly_rent_price'], dtype='object')

In [21]:
y = df[['sample_nightly_rent_price']]
X = df[['current_monthly_rent','num_bedrooms','percentile_10th_price', 'percentile_90th_price']]

In [22]:
X = sm.add_constant(X)

In [23]:
model = sm.OLS(y,X).fit()

In [24]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     sample_nightly_rent_price   R-squared:                       0.838
Model:                                   OLS   Adj. R-squared:                  0.838
Method:                        Least Squares   F-statistic:                 1.927e+04
Date:                       Thu, 09 Dec 2021   Prob (F-statistic):               0.00
Time:                               14:12:36   Log-Likelihood:                -85816.
No. Observations:                      14888   AIC:                         1.716e+05
Df Residuals:                          14883   BIC:                         1.717e+05
Df Model:                                  4                                         
Covariance Type:                   nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    15.0587      2.257      6.673      0.000      10.636      19.482
current_monthly_rent      0.0004      0.001      0.749      0.454      -0.001       0.001
num_bedrooms             -1.3017      1.379     -0.944      0.345      -4.005       1.401
percentile_10th_price     0.5275      0.010     50.610      0.000       0.507       0.548
percentile_90th_price     0.4589      0.003    175.019      0.000       0.454       0.464
==============================================================================
Omnibus:                     1054.485   Durbin-Watson:                   2.100
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5088.129
Skew:                          -0.147   Prob(JB):                         0.00
Kurtosis:                       5.849   Cond. No.                     9.43e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.43e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [25]:
prediction = model.predict(X)

In [26]:
prediction

0       281.38
1       225.36
2       277.98
3       306.13
4       314.38
         ...  
14883   496.57
14884   176.19
14885   240.35
14886   240.81
14887   200.09
Length: 14888, dtype: float64

#### Python code done by Dennis Lam